# KBO 경기별 투수 데이터 전처리

이 노트북은 투수별 매치 데이터에 상대팀(Opponent) 정보를 추가하고,
**경기 단위**로 데이터를 재구조화합니다.

## 목표
1. `data/gamelog/` 경기 일정 데이터에서 상대팀 정보 추출
2. `data/raw/matches/` 투수 기록에 상대팀 필드 추가
3. **경기별**(양 팀 투수 기록을 하나로) 그룹화하여 저장
4. 전처리된 데이터를 `data/processed/matches/`에 저장

## 1. 필수 라이브러리 및 경로 설정

In [ ]:
import json
import csv
from pathlib import Path
from typing import Dict, List, Any, Tuple
from collections import defaultdict

# 경로 설정
BASE_DIR = Path(".")
GAMELOG_DIR = BASE_DIR / "data" / "gamelog"
RAW_MATCHES_DIR = BASE_DIR / "data" / "raw" / "matches"
PROCESSED_DIR = BASE_DIR / "data" / "processed" / "matches"

# 출력 디렉토리 생성
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# 팀 매핑 (한글 -> 영문 정규화)
# utils.py의 TEAM_MAP을 여기서 정의 (의존성 제거를 위해)
TEAM_MAP: Dict[str, str] = {
    # 한화 이글스
    "한화": "Hanwha", "한화 이글스": "Hanwha", "한화이글스": "Hanwha",
    "이글스": "Hanwha", "Hanwha Eagles": "Hanwha", "hanwha": "Hanwha", "Hanwha": "Hanwha",
    
    # LG 트윈스
    "LG": "LG", "엘지": "LG", "LG 트윈스": "LG", "LG트윈스": "LG",
    "트윈스": "LG", "LG Twins": "LG",
    
    # 삼성 라이온즈
    "삼성": "Samsung", "삼성 라이온즈": "Samsung", "삼성라이온즈": "Samsung",
    "라이온즈": "Samsung", "Samsung Lions": "Samsung", "samsung": "Samsung", "Samsung": "Samsung",
    
    # 두산 베어스
    "두산": "Doosan", "두산 베어스": "Doosan", "두산베어스": "Doosan",
    "베어스": "Doosan", "Doosan Bears": "Doosan", "doosan": "Doosan", "Doosan": "Doosan",
    
    # 롯데 자이언츠
    "롯데": "Lotte", "롯데 자이언츠": "Lotte", "롯데자이언츠": "Lotte",
    "자이언츠": "Lotte", "Lotte Giants": "Lotte", "lotte": "Lotte", "Lotte": "Lotte",
    
    # 기아 타이거즈
    "기아": "KIA", "KIA": "KIA", "기아 타이거즈": "KIA", "기아타이거즈": "KIA",
    "타이거즈": "KIA", "KIA Tigers": "KIA", "kia": "KIA",
    
    # NC 다이노스
    "NC": "NC", "엔씨": "NC", "NC 다이노스": "NC", "NC다이노스": "NC",
    "다이노스": "NC", "NC Dinos": "NC", "nc": "NC",
    
    # SSG 랜더스
    "SSG": "SSG", "에스에스지": "SSG", "SSG 랜더스": "SSG", "SSG랜더스": "SSG",
    "랜더스": "SSG", "인천": "SSG", "SSG Landers": "SSG", "ssg": "SSG",
    
    # 키움 히어로즈
    "키움": "Kiwoom", "키움 히어로즈": "Kiwoom", "키움히어로즈": "Kiwoom",
    "히어로즈": "Kiwoom", "Kiwoom Heroes": "Kiwoom", "kiwoom": "Kiwoom", "Kiwoom": "Kiwoom",
    
    # KT 위즈
    "KT": "KT", "케이티": "KT", "KT 위즈": "KT", "KT위즈": "KT",
    "위즈": "KT", "KT Wiz": "KT", "kt": "KT",
}

print("✅ 라이브러리 및 경로 설정 완료")
print(f"   - 경기 로그 디렉토리: {GAMELOG_DIR}")
print(f"   - 원본 매치 데이터: {RAW_MATCHES_DIR}")
print(f"   - 처리된 데이터 저장: {PROCESSED_DIR}")

## 2. 경기 일정 데이터 로드 (정규시즌 + 포스트시즌)

`data/gamelog/` 폴더에서 정규시즌과 포스트시즌 경기 일정을 로드합니다.

In [ ]:
def load_game_schedules() -> List[Dict[str, Any]]:
    """
    정규시즌과 포스트시즌 경기 일정을 모두 로드합니다.
    
    Returns:
        List[Dict]: 경기 일정 리스트 (date, home_team, away_team, season_type, ...)
    """
    schedules = []
    
    # 정규시즌 로드
    regular_path = GAMELOG_DIR / "games_2025.csv"
    if regular_path.exists():
        with open(regular_path, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                schedules.append({
                    "date": row['date'],
                    "home_team": row['home_team'],
                    "away_team": row['away_team'],
                    "home_runs": row.get('home_runs', ''),
                    "away_runs": row.get('away_runs', ''),
                    "season_type": "Regular"
                })
        print(f"✅ 정규시즌 경기 로드: {len([s for s in schedules if s['season_type']=='Regular'])}경기")
    
    # 포스트시즌 로드
    post_path = GAMELOG_DIR / "games_2025_postseason.csv"
    if post_path.exists():
        with open(post_path, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                schedules.append({
                    "date": row['date'],
                    "home_team": row['home_team'],
                    "away_team": row['away_team'],
                    "home_runs": row.get('home_runs', ''),
                    "away_runs": row.get('away_runs', ''),
                    "season_type": "Post",
                    "stage": row.get('stage', 'Postseason')
                })
        print(f"✅ 포스트시즌 경기 로드: {len([s for s in schedules if s['season_type']=='Post'])}경기")
    
    return schedules

# 경기 일정 로드
game_schedules = load_game_schedules()
print(f"\n📊 총 {len(game_schedules)}경기 일정 로드 완료")

# 샘플 확인
print("\n[샘플 데이터 - 처음 3경기]")
for game in game_schedules[:3]:
    print(f"  {game['date']}: {game['home_team']} vs {game['away_team']} ({game['season_type']})")

## 3. (날짜, 팀) → 상대팀 매핑 딕셔너리 생성

각 경기에서 특정 팀이 어떤 팀과 경기했는지 양방향 매핑을 생성합니다.  
또한, **경기 고유 ID(game_id)** 매핑도 함께 생성합니다.

In [ ]:
def normalize_team(team_name: str) -> str:
    """팀명을 정규화합니다."""
    return TEAM_MAP.get(team_name, team_name)


def build_game_mappings(schedules: List[Dict]) -> Tuple[Dict[str, str], Dict[str, Dict], Dict[str, str]]:
    """
    경기 일정에서 세 가지 매핑을 생성합니다.
    
    Returns:
        opponent_map: (날짜_팀) -> 상대팀 (양방향)
        game_info_map: game_id -> 경기 정보 (home_team, away_team, date, season_type 등)
        team_to_game_id: (날짜_팀) -> game_id
    """
    opponent_map = {}      # "2025-03-22_LG" -> "Lotte"
    game_info_map = {}     # "2025-03-22_LG_vs_Lotte" -> {경기 정보}
    team_to_game_id = {}   # "2025-03-22_LG" -> "2025-03-22_LG_vs_Lotte"
    
    for game in schedules:
        date = game['date']
        home_raw = game['home_team']
        away_raw = game['away_team']
        
        # 팀명 정규화
        home = normalize_team(home_raw)
        away = normalize_team(away_raw)
        
        # 경기 고유 ID 생성 (홈팀 기준으로 통일)
        game_id = f"{date}_{home}_vs_{away}"
        
        # 1. 상대팀 매핑 (양방향)
        opponent_map[f"{date}_{home}"] = away
        opponent_map[f"{date}_{away}"] = home
        
        # 2. 경기 정보 매핑
        game_info_map[game_id] = {
            "game_id": game_id,
            "date": date,
            "home_team": home,
            "away_team": away,
            "home_runs": game.get('home_runs', ''),
            "away_runs": game.get('away_runs', ''),
            "season_type": game.get('season_type', 'Regular'),
            "stage": game.get('stage', '')
        }
        
        # 3. 팀 -> game_id 매핑 (양방향)
        team_to_game_id[f"{date}_{home}"] = game_id
        team_to_game_id[f"{date}_{away}"] = game_id
    
    return opponent_map, game_info_map, team_to_game_id


# 매핑 생성
opponent_map, game_info_map, team_to_game_id = build_game_mappings(game_schedules)

print(f"✅ 매핑 생성 완료")
print(f"   - 상대팀 매핑: {len(opponent_map)}개 (양방향)")
print(f"   - 경기 정보: {len(game_info_map)}경기")
print(f"   - 팀→경기ID 매핑: {len(team_to_game_id)}개")

# 샘플 확인
print("\n[샘플 - 상대팀 매핑]")
sample_keys = list(opponent_map.keys())[:4]
for key in sample_keys:
    print(f"  {key} -> {opponent_map[key]}")

print("\n[샘플 - 경기 정보]")
sample_game = list(game_info_map.values())[0]
print(f"  {sample_game}")

## 4. 원본 투수 매치 데이터 로드

`data/raw/matches/` 디렉토리에서 JSON 파일들을 로드합니다.

In [ ]:
def load_raw_match_data() -> List[Dict[str, Any]]:
    """
    원본 투수 매치 데이터를 로드합니다.
    
    Returns:
        List[Dict]: 개별 투수 레코드 리스트
    """
    all_records = []
    
    json_files = list(RAW_MATCHES_DIR.glob("*.json"))
    print(f"📂 발견된 JSON 파일: {len(json_files)}개")
    
    for json_file in json_files:
        # 파일명에서 시즌 타입 추출
        filename = json_file.stem.upper()
        is_postseason = "POST" in filename
        season_type = "Post" if is_postseason else "Regular"
        
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                content = json.load(f)
            
            # JSON 구조: [ { "dataset_id": ..., "data": [...] }, ... ]
            if isinstance(content, list):
                for dataset in content:
                    records = dataset.get('data', [])
                    
                    for record in records:
                        # 메타데이터 추가
                        record['_source_file'] = json_file.name
                        record['_season_type'] = season_type
                        all_records.append(record)
                    
                    print(f"   - {json_file.name}: {len(records)}개 레코드")
            
        except Exception as e:
            print(f"❌ 오류 ({json_file.name}): {e}")
    
    return all_records


# 원본 데이터 로드
raw_records = load_raw_match_data()
print(f"\n📊 총 {len(raw_records)}개의 투수 레코드 로드 완료")

# 샘플 확인
print("\n[샘플 레코드]")
if raw_records:
    sample = raw_records[0]
    print(f"  Date: {sample.get('Date')}")
    print(f"  Team: {sample.get('Team')}")
    print(f"  Name: {sample.get('Name')}")
    print(f"  Result: {sample.get('Result')}")
    print(f"  IP: {sample.get('IP')}, ER: {sample.get('ER')}")

## 5. 각 투수 레코드에 상대팀(Opponent) 정보 추가

매핑 딕셔너리를 사용해 각 레코드에 상대팀 필드를 추가합니다.

In [ ]:
def add_opponent_info(records: List[Dict], opp_map: Dict[str, str], game_id_map: Dict[str, str]) -> List[Dict]:
    """
    각 투수 레코드에 상대팀 정보와 game_id를 추가합니다.
    
    Args:
        records: 투수 레코드 리스트
        opp_map: (날짜_팀) -> 상대팀 매핑
        game_id_map: (날짜_팀) -> game_id 매핑
    
    Returns:
        상대팀 정보가 추가된 레코드 리스트
    """
    missing_count = 0
    
    for record in records:
        date = record.get('Date', '')
        team_raw = record.get('Team', '')
        team = normalize_team(team_raw)
        
        key = f"{date}_{team}"
        
        # 상대팀 조회
        opponent = opp_map.get(key, None)
        game_id = game_id_map.get(key, None)
        
        if opponent:
            record['Opponent'] = opponent
            record['game_id'] = game_id
            # 팀명도 정규화된 버전으로 업데이트
            record['Team_normalized'] = team
        else:
            record['Opponent'] = 'Unknown'
            record['game_id'] = None
            record['Team_normalized'] = team
            missing_count += 1
    
    return records, missing_count


# 상대팀 정보 추가
enriched_records, missing = add_opponent_info(raw_records, opponent_map, team_to_game_id)

print(f"✅ 상대팀 정보 추가 완료")
print(f"   - 성공: {len(enriched_records) - missing}개")
print(f"   - 매핑 실패: {missing}개")

# 샘플 확인
print("\n[샘플 - 상대팀 추가된 레코드]")
for record in enriched_records[:3]:
    print(f"  {record.get('Date')} | {record.get('Team')} vs {record.get('Opponent')} | {record.get('Name')}")
    print(f"    game_id: {record.get('game_id')}")

## 6. 경기별(game_id)로 레코드 그룹화

양 팀 투수 기록을 하나의 경기 문서로 통합합니다.  
예: 2025-04-17 롯데 vs LG 경기 → 롯데 투수들 + LG 투수들을 하나의 문서로

In [ ]:
def group_by_game(records: List[Dict], game_info: Dict[str, Dict]) -> List[Dict[str, Any]]:
    """
    투수 레코드를 경기별(game_id)로 그룹화합니다.
    
    Args:
        records: 상대팀 정보가 추가된 투수 레코드 리스트
        game_info: game_id -> 경기 정보 매핑
    
    Returns:
        경기별 그룹화된 데이터 리스트
    """
    # game_id별로 레코드 그룹화
    games = defaultdict(lambda: {
        "home_pitchers": [],
        "away_pitchers": []
    })
    
    skipped = 0
    
    for record in records:
        game_id = record.get('game_id')
        if not game_id:
            skipped += 1
            continue
        
        # 해당 경기 정보 조회
        ginfo = game_info.get(game_id)
        if not ginfo:
            skipped += 1
            continue
        
        team = record.get('Team_normalized', record.get('Team'))
        home_team = ginfo['home_team']
        away_team = ginfo['away_team']
        
        # 홈/원정 구분하여 저장
        if team == home_team:
            games[game_id]["home_pitchers"].append(record)
        elif team == away_team:
            games[game_id]["away_pitchers"].append(record)
        else:
            # 팀 매칭 실패 - 로깅 후 원정으로 분류
            print(f"⚠️ 팀 매칭 실패: {team} not in ({home_team}, {away_team})")
            games[game_id]["away_pitchers"].append(record)
    
    # 최종 경기별 문서 생성
    game_documents = []
    
    for game_id, pitchers in games.items():
        ginfo = game_info.get(game_id, {})
        
        game_doc = {
            "game_id": game_id,
            "date": ginfo.get('date', ''),
            "home_team": ginfo.get('home_team', ''),
            "away_team": ginfo.get('away_team', ''),
            "home_runs": ginfo.get('home_runs', ''),
            "away_runs": ginfo.get('away_runs', ''),
            "season_type": ginfo.get('season_type', 'Regular'),
            "stage": ginfo.get('stage', ''),
            "home_pitchers": pitchers["home_pitchers"],
            "away_pitchers": pitchers["away_pitchers"],
            "home_pitcher_count": len(pitchers["home_pitchers"]),
            "away_pitcher_count": len(pitchers["away_pitchers"]),
            "total_pitcher_count": len(pitchers["home_pitchers"]) + len(pitchers["away_pitchers"])
        }
        
        game_documents.append(game_doc)
    
    # 날짜순 정렬
    game_documents.sort(key=lambda x: x['date'])
    
    print(f"⚠️ 스킵된 레코드: {skipped}개")
    
    return game_documents


# 경기별 그룹화
game_documents = group_by_game(enriched_records, game_info_map)

print(f"\n✅ 경기별 그룹화 완료")
print(f"   - 총 경기 수: {len(game_documents)}")

# 통계
total_pitchers = sum(g['total_pitcher_count'] for g in game_documents)
avg_pitchers = total_pitchers / len(game_documents) if game_documents else 0
print(f"   - 총 투수 레코드: {total_pitchers}")
print(f"   - 경기당 평균 투수: {avg_pitchers:.1f}명")

# 샘플 확인
print("\n[샘플 - 경기별 문서]")
if game_documents:
    sample_game = game_documents[0]
    print(f"  game_id: {sample_game['game_id']}")
    print(f"  {sample_game['date']}: {sample_game['home_team']} vs {sample_game['away_team']}")
    print(f"  홈 투수 {sample_game['home_pitcher_count']}명, 원정 투수 {sample_game['away_pitcher_count']}명")
    print(f"  홈 투수 명단: {[p['Name'] for p in sample_game['home_pitchers'][:3]]}...")
    print(f"  원정 투수 명단: {[p['Name'] for p in sample_game['away_pitchers'][:3]]}...")

## 7. 전처리된 JSON 데이터 저장

경기별로 그룹화된 데이터를 `data/processed/matches/`에 저장합니다.

In [ ]:
def save_processed_data(game_docs: List[Dict], output_dir: Path):
    """
    전처리된 경기별 데이터를 JSON 파일로 저장합니다.
    
    정규시즌과 포스트시즌을 별도 파일로 저장합니다.
    """
    # 시즌 타입별 분리
    regular_games = [g for g in game_docs if g['season_type'] == 'Regular']
    post_games = [g for g in game_docs if g['season_type'] == 'Post']
    
    # 정규시즌 저장
    if regular_games:
        regular_output = {
            "dataset_id": "2025_REGULAR_GAME_PITCHING_STATS",
            "name": "2025 Regular Season Game-level Pitching Data",
            "description": "경기별(양팀 투수 기록 통합) 투수 데이터",
            "total_games": len(regular_games),
            "data": regular_games
        }
        
        regular_path = output_dir / "2025_REGULAR_GAME_PITCHING_STATS.json"
        with open(regular_path, 'w', encoding='utf-8') as f:
            json.dump([regular_output], f, ensure_ascii=False, indent=2)
        print(f"✅ 정규시즌 저장: {regular_path}")
        print(f"   - {len(regular_games)}경기")
    
    # 포스트시즌 저장
    if post_games:
        post_output = {
            "dataset_id": "2025_POST_GAME_PITCHING_STATS",
            "name": "2025 Postseason Game-level Pitching Data",
            "description": "경기별(양팀 투수 기록 통합) 투수 데이터",
            "total_games": len(post_games),
            "data": post_games
        }
        
        post_path = output_dir / "2025_POST_GAME_PITCHING_STATS.json"
        with open(post_path, 'w', encoding='utf-8') as f:
            json.dump([post_output], f, ensure_ascii=False, indent=2)
        print(f"✅ 포스트시즌 저장: {post_path}")
        print(f"   - {len(post_games)}경기")


# 데이터 저장
save_processed_data(game_documents, PROCESSED_DIR)

print(f"\n📁 저장 위치: {PROCESSED_DIR}")

## 8. 전처리 결과 검증

저장된 데이터를 다시 로드하여 구조를 검증합니다.

In [ ]:
def verify_processed_data(output_dir: Path):
    """전처리된 데이터를 로드하여 검증합니다."""
    
    json_files = list(output_dir.glob("*.json"))
    
    if not json_files:
        print("⚠️ 저장된 파일이 없습니다.")
        return
    
    total_games = 0
    total_pitchers = 0
    
    for json_file in json_files:
        print(f"\n📄 파일: {json_file.name}")
        
        with open(json_file, 'r', encoding='utf-8') as f:
            content = json.load(f)
        
        for dataset in content:
            games = dataset.get('data', [])
            print(f"   - 경기 수: {len(games)}")
            
            for game in games:
                total_games += 1
                total_pitchers += game.get('total_pitcher_count', 0)
            
            # 샘플 경기 상세 출력
            if games:
                sample = games[0]
                print(f"\n   [샘플 경기 상세]")
                print(f"   game_id: {sample['game_id']}")
                print(f"   날짜: {sample['date']}")
                print(f"   대진: {sample['home_team']} (홈) vs {sample['away_team']} (원정)")
                print(f"   스코어: {sample['home_runs']} - {sample['away_runs']}")
                print(f"   시즌: {sample['season_type']}")
                
                print(f"\n   홈팀 투수 ({sample['home_pitcher_count']}명):")
                for p in sample['home_pitchers'][:3]:
                    result = p.get('Result', '-')
                    print(f"      - {p['Name']}: {p.get('IP')}이닝, {p.get('ER')}자책, 결과: {result}")
                if sample['home_pitcher_count'] > 3:
                    print(f"      ... 외 {sample['home_pitcher_count'] - 3}명")
                
                print(f"\n   원정팀 투수 ({sample['away_pitcher_count']}명):")
                for p in sample['away_pitchers'][:3]:
                    result = p.get('Result', '-')
                    print(f"      - {p['Name']}: {p.get('IP')}이닝, {p.get('ER')}자책, 결과: {result}")
                if sample['away_pitcher_count'] > 3:
                    print(f"      ... 외 {sample['away_pitcher_count'] - 3}명")
    
    print(f"\n" + "="*50)
    print(f"📊 전체 통계")
    print(f"   - 총 경기 수: {total_games}")
    print(f"   - 총 투수 레코드: {total_pitchers}")
    print(f"   - 경기당 평균 투수: {total_pitchers/total_games:.1f}명")


# 검증 실행
verify_processed_data(PROCESSED_DIR)

## ✅ 전처리 완료

전처리된 데이터가 `data/processed/matches/`에 저장되었습니다.

### 다음 단계

1. **main.ipynb 실행**: `ingest_all_data()` 함수가 자동으로 전처리된 데이터를 로드합니다.
2. 데이터 적재 후 쿼리 테스트 진행

### 파이프라인 흐름

```
┌─────────────────────────────────────────────────────────────────────────┐
│  1. preprocess.ipynb (전처리)                                            │
│     - data/raw/matches/*.json 로드                                       │
│     - data/gamelog/*.csv에서 상대팀 정보 매핑                              │
│     - 경기별(양팀 투수 통합) 그룹화                                         │
│     - data/processed/matches/*.json 저장                                 │
└─────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────┐
│  2. main.ipynb (적재 및 검색)                                             │
│     - ingest_all_data() → data/processed/matches/ 로드                   │
│     - generate_game_description() → 자연어 설명 생성                       │
│     - ChromaDB에 임베딩 및 저장                                           │
│     - RAG 기반 쿼리 응답                                                  │
└─────────────────────────────────────────────────────────────────────────┘
```

### 문서 구조 예시

각 경기가 하나의 문서로 저장됩니다:

```
game_id: "2025-04-17_LG_vs_Lotte"
date: "2025-04-17"
home_team: "LG"
away_team: "Lotte"
home_runs: "3"
away_runs: "7"
season_type: "Regular"
home_pitchers: [이민호, 김진성, ...]
away_pitchers: [반즈, 송재영, ...]
```

description 예시:
> "2025년 04월 17일 Regular 시즌 경기: LG(홈) vs Lotte(원정). 스코어 3-7 (Lotte 승리). 
> LG 투수진(8이닝 7자책): 이민호(패), 김진성, 고우석. 
> Lotte 투수진(9이닝 3자책): 반즈(승), 송재영, 박준우."